In [33]:
b=2
import pandas as pd
import os
import logging
from collections import Counter

logging.basicConfig(level=logging.DEBUG,format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("hi")








2025-08-09 16:04:11,434 - INFO - hi


In [34]:
import pandas as pd
import logging

def parse_tree_file(tree_df,children,parents):
    # create a dictionary from taxonomic id to index of corresponding row
    info = {}
    for i in range(len(children)):
        info[int(children[i])] = int(i)

    # Convert to tree structure (dictionary of sets)
    Tree = {}
    for i in range(len(tree_df[0])):
        parent = int(parents[i])
        child = int(children[i])
        if parent in Tree:
            Tree[parent].add(child)
        else:
            Tree[parent] = {child}
    return info, Tree 


def find_tax2root(info,parents, tid):
    if tid not in info:
        #print(tid)
        return -1
    list_tax2root=[]
    while tid != 1:    
        list_tax2root.append(tid)
        tid = int(parents[info[tid]])
    list_tax2root.append(tid)
    return list_tax2root



def get_tax_info(tree_file,tax_genome_file):
    logging.debug("Tree file: "+ tree_file)
    tree_df = pd.read_csv(tree_file, sep="\t", header=None)

    logging.debug("Read the tree file") 
    children = tree_df[0]
    parents = tree_df[2]
    info, Tree = parse_tree_file(tree_df,children,parents)
    logging.debug("Parsed the tree file. Size of info: "+str(   len(info))+"Size of Tree: "+str(len(Tree)))


    tax_genome_f = open(tax_genome_file,'r') # genomes in the kraken index  
    tax_genome = set()
    for line in tax_genome_f:
        tax_genome.add(int(line.strip()))
    logging.debug('Number of genomes/strain '+str(len(tax_genome))+" in kraken index")

    tax_index_=[]
    tax_notfound=[]
    for tax_ in tax_genome:
        list_tax2root = find_tax2root(info,parents, tax_)
        if list_tax2root!=-1:
            tax_index_ +=list_tax2root
        else:
            tax_notfound.append(tax_)
    tax_index =set(tax_index_) # all the tax in genomes of kraken, plus all up to root 
    logging.debug('Number of tax ids'+str(len(tax_index))+" "+str(len(tax_index_))) # 
    logging.debug("Genome tax not found in kraken tree"+str(tax_notfound))

    return info, Tree, tax_index, tax_genome, parents, tree_df


def get_tax2path(tax_genome, info, parents):

    tax2root_all=[]
    tax2root_all_dic={} # paths endint at all levels 
    # paths endint at leaf level (no Vague positive )
    tax_index_list= list(tax_genome)  # tax_genome_specieslevel 
    for tax in tax_index_list:   
        tax2root= find_tax2root(info, parents, tax)
        if tax2root==-1:
            logging.debug("tax not found in kraken tree: "+str(tax))
        else:    #elif 2 in tax2root: # selecting only bacteria
            tax2root_all.append(tax2root) # # [655353, 655352, 655351, 356, 28211, 1224, 3379134, 2, 131567, 1]
            tax2root_all_dic[tax]=tax2root
    logging.debug("number of paths"+str(len(tax2root_all)))
    tax2path = {} # a tax is present in which paths
    tax2depth = {} 
    for tax2root_path_idx, tax2root_path in enumerate(tax2root_all):
        for tax_idx,tax in enumerate(tax2root_path[::-1]):
            tax2depth[tax]=tax_idx
            if tax in tax2path:
                tax2path[tax].append(tax2root_path_idx) #tax2root_path[0]
            else:
                tax2path[tax]=[tax2root_path_idx]
    logging.debug("number of tax2path"+str(len(tax2path))+" "+str(len(tax2depth)))

    return tax2path, tax2depth




def find_tax_level(info,tree_df,parents, tid, tax_level="species"):
    if tid not in info:
        #print(tid)
        return -1
    while tid != 1:    
        tid_row = info[tid]
        if tree_df[4][tid_row] == tax_level:
            break
        else:
            tid = int(parents[info[tid]]) #find_parent_node(tid)
            # if retrun 1, it means that the tid is more towaards root that the requested level
    return tid



def lca_finder_list(info,parents, tax_list):
    lca=tax_list[0]
    
    for tax2 in tax_list[1:]:
        lca = lca_finder(info,parents, lca,tax2)
        
    return lca

def lca_finder(info,parents, tax1,tax2):
    list_tax2root_1 = find_tax2root(info,parents, tax1)
    list_tax2root_2 = find_tax2root(info,parents, tax2)
    
    list_tax2root1 =list_tax2root_1 [::-1] # reverse from root to leaf
    list_tax2root2 =list_tax2root_2 [::-1]
    
    depth_i = 0
    while(depth_i < len(list_tax2root1) and depth_i < len(list_tax2root2)):
        #print(depth_i)
        if list_tax2root1[depth_i] != list_tax2root2[depth_i]:
            break
        depth_i += 1
    assert list_tax2root1[depth_i-1] == list_tax2root2[depth_i-1], "issue in LCA finding"
    lca=list_tax2root1[depth_i-1]
    #print(lca)
    return lca


def read_truth_file(truth_file):
    merged = pd.read_csv(truth_file, names=["read_name", "taxon"])
    dic_tax_truth={}
    for index, row in merged.iterrows():
        dic_tax_truth[row['read_name']]=row['taxon']
    return dic_tax_truth



In [ ]:
workingdir="/vast/blangme2/smajidi5/metagenomics/metakpick_project/MetaKPick/"
in_folder= workingdir+"../files/"
tree_file=in_folder+"nodes.dmp"
tax_genome_file= in_folder + "seqid2taxid.map_tax_uniq"

info, Tree, tax_index, tax_genome, parents, tree_df = get_tax_info(tree_file,tax_genome_file)    
tax2path, tax2depth = get_tax2path(tax_genome, info, parents)

folder_input =workingdir+"../../changek/simulatation/"
truth_file=folder_input+"true_tax.csv" # generated by generate_training.ipynb


2025-08-09 16:01:42,823 - DEBUG - Tree file: /vast/blangme2/smajidi5/metagenomics/metakpick_project/MetaKPick/../files/nodes.dmp
2025-08-09 16:01:49,705 - DEBUG - Read the tree file
2025-08-09 16:02:02,178 - DEBUG - Parsed the tree file. Size of info: 2667703Size of Tree: 241610
2025-08-09 16:02:02,188 - DEBUG - Number of genomes/strain 33646 in kraken index
2025-08-09 16:02:02,709 - DEBUG - Number of tax ids52229 353816
2025-08-09 16:02:02,710 - DEBUG - Genome tax not found in kraken tree[3024248, 2778655, 386490]
2025-08-09 16:02:02,766 - DEBUG - tax not found in kraken tree: 3024248
2025-08-09 16:02:02,831 - DEBUG - tax not found in kraken tree: 2778655
2025-08-09 16:02:03,198 - DEBUG - tax not found in kraken tree: 386490
2025-08-09 16:02:03,221 - DEBUG - number of paths33643
2025-08-09 16:02:03,301 - DEBUG - number of tax2path52229 52229


NameError: name 'read_truth_file' is not defined

In [35]:

dic_tax_truth = read_truth_file(truth_file)
logging.info("Number of reads in the truth file: "+str(len(dic_tax_truth)))

classification_folder=folder_input+"classification/" 
cases=[i.split("_")[0] for i in os.listdir(classification_folder) if i.endswith('_out')] #["k"+str(k) for k in range(15,32)]


2025-08-09 16:04:53,798 - INFO - Number of reads in the truth file: 248875


In [2]:

def read_kraken_file(kraken_file):

    # "C"/"U": a one letter code indicating that the sequence was either classified or unclassified.
    # The sequence ID, obtained from the FASTA/FASTQ header.
    # The taxonomy ID Kraken 2 used to label the sequence; this is 0 if the sequence is unclassified.
    # The length of the sequence in bp. In the case of paired read data, this will be a string containing the lengths of the two sequences in bp, separated by a pipe character, e.g. "98|94".
    # A space-delimited list indicating the LCA mapping of each  k-mer in the sequence(s).
    # For example, "562:13 561:4 A:31 0:1 562:3" would indicate that:
    # the first 13 k-mers mapped to taxonomy ID #562 ... 31  ambiguous nucleotide (A)
    file_o=open(kraken_file,'r')
    dic_kraken={}
    for line in file_o:
        line_split=line.strip().split("\t")
        classified ,read_id, tax_krak, read_len, tax_kmers = line_split
        if classified=="C":
            
            tax_kmer_dic = {}
            tax_kmer_num_dic={}
            pos=-1
            for tax_kmer in tax_kmers.split(" "):
                tax_, num_raw =  tax_kmer.split(":")
                num=int(num_raw)
                pos+=num     # how many consecutive kmer has this taxid # 0:6 543:1 91347:1 six unclassified, 
                if tax_ =='0' or tax_ =='A':
                    continue
                tax=int(tax_)
                if tax in tax_kmer_dic:
                    tax_kmer_dic[tax].append( (pos,num))
                    tax_kmer_num_dic[tax]+=num
                else:    
                    tax_kmer_dic[tax]=[(pos,num)]
                    tax_kmer_num_dic[tax]=num
            kraken_read_info = int(tax_krak), int(read_len), tax_kmer_dic, tax_kmer_num_dic
        elif classified=="U":
            read_id, tax_krak, read_len, tax_kmers = line_split[1:]
            kraken_read_info = 0, int(read_len), {}, {}
        else:
            print("Error: ",line_split)
            kraken_read_info = 0, 0, {}, {}
        
        dic_kraken[read_id] = kraken_read_info 
    file_o.close()
    return dic_kraken

In [3]:
kraken_file="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/k31_out"
dic_kraken=read_kraken_file(kraken_file)
#print(dic_kraken)

In [4]:
len(dic_kraken)

249069

In [ ]:
dic_kraken['NZ_OZ243912.1__1_1']

(1885344,
 1253,
 {1885344: [(558, 2),
   (646, 7),
   (675, 2),
   (706, 3),
   (840, 13),
   (897, 8),
   (938, 6),
   (1033, 27),
   (1173, 7)]},
 {1885344: 75})

In [8]:
classification_folder="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/"

cases=[i.split("_")[0] for i in os.listdir(classification_folder) if i.endswith('_out')] #["k"+str(k) for k in range(15,32)]
cases=cases[4:5]+cases[-1:]
cases

['k25', 'k31']

In [ ]:

def read_kraken_all(cases, classification_folder): # , readids_max, num_reads=10000
    print("read kraken's k-mer  count per tax")
    #folder="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/" # 
    #cases=['k19','k25','k31'] # ,'k21'
    kraken_kmers_cases={}
    for case in reversed(cases): 
        print(case)
        kraken_kmers_cases[case]=  read_kraken_file(classification_folder+case+"_out")
        logging.info(case+" "+str(len(kraken_kmers_cases[case])))
    
    for case_k, case in  enumerate(cases): 
        read_names_case=set(kraken_kmers_cases[case].keys())
        if case_k==0:
            read_names_case0= read_names_case
        else:
            assert read_names_case0 == read_names_case, "we expect the same reads in all cases"+case+" is not the same as "+cases[0]+str(len(read_names_case0))+" "+str(len(read_names_case)) 

    return read_names_case0, kraken_kmers_cases



In [17]:
read_names_list, kraken_kmers_cases = read_kraken_all(cases, classification_folder)

read kraken's k-mer  count per tax
k31


2025-08-09 15:51:37,953 - INFO - k31 249069


k25


2025-08-09 15:52:02,213 - INFO - k25 249069


In [20]:
kraken_kmers_cases.keys()

dict_keys(['k31', 'k25'])

In [21]:
kraken_kmers_cases.keys()

dict_keys(['k31', 'k25'])

In [27]:
kraken_kmers_cases['k31']['NZ_OZ243912.1__1_1'][0], kraken_kmers_cases['k31']['NZ_RREV01000078.1__1_1'][0]  

(1885344, 0)

In [ ]:

def get_tax_depth(kraken_kmers_cases, info, parents):
    logging.debug("Getting the tax depth")
    read_tax_depth={}
    for case, kraken_kmers in kraken_kmers_cases.items(): 
        read_tax_depth[case]={}
        for read_name, kraken_read_info in kraken_kmers.items():
            reported_tax=kraken_read_info[0]
            tax2root_list = find_tax2root(info,parents, reported_tax) # [2711156, 1649486, 41297, 204457, 28211, 1224, 3379134, 2, 131567, 1]
            if tax2root_list!=-1:
                depth=len(tax2root_list)
            else:
                depth=0
            read_tax_depth[case][read_name]=depth
            
    logging.debug("Number of reads in read_tax_depth: "+str(len(read_tax_depth))+" "+str(len(read_tax_depth[case])))
    logging.debug("Counter of read_tax_depth: "+str(Counter(read_tax_depth[case].values())))

    return read_tax_depth  

In [41]:
read_tax_depth = get_tax_depth(kraken_kmers_cases, info,parents)

2025-08-09 16:06:13,705 - DEBUG - Getting the tax depth
2025-08-09 16:06:19,684 - DEBUG - Number of reads in read_tax_depth: 2 249069
2025-08-09 16:06:19,700 - DEBUG - Counter of read_tax_depth: Counter({10: 86765, 11: 54985, 12: 27308, 0: 26623, 9: 16158, 8: 13085, 7: 6156, 13: 5319, 6: 5238, 1: 2278, 3: 1684, 5: 1207, 32: 1028, 4: 581, 14: 514, 2: 132, 15: 6, 16: 1, 22: 1})


In [ ]:
read_tax_depth['k31']['NZ_OZ243912.1__1_1']


13

In [47]:
find_tax2root(info,parents, 1885344) , len(find_tax2root(info,parents, 1885344) )

([1885344,
  2640460,
  1434037,
  1853227,
  1853224,
  1853222,
  1853220,
  68336,
  1783270,
  3379134,
  2,
  131567,
  1],
 13)

In [ ]:

import pandas as pd
in_folder= "../../files/"
tree_file=in_folder+"nodes.dmp"
tax_genome_file= in_folder + "seqid2taxid.map_tax_uniq"

info, Tree, tax_index, tax_genome = get_tax_info(tree_file,tax_genome_file)

In [48]:
len(dic_tax_truth)

248875

In [ ]:

tax2path, tax2depth = get_tax2path(tax_genome, info, parents)

In [ ]:

def get_tax_depth(kraken_kmers_cases, info, parents):
    logging.debug("Getting the tax depth")
    read_tax_depth={}
    for case, kraken_kmers in kraken_kmers_cases.items(): 
        read_tax_depth[case]={}
        for read_name, kraken_read_info in kraken_kmers.items():
            reported_tax=kraken_read_info[0]


In [85]:



def calculate_tp_fp(mode,input_dic,dic_tax_truth,info,tree_df,parents,tax_level,tax_index):

    """
    input_dic:
        raw_kraken: kraken_kmers_case     mode:"raw_kraken"
        predicted_tax: predicted_tax_dic  mode:"predicted_tax"
    """
    read_tpfp_dic={"TP":set(),"FP-level-notindex":set(), 'FP-higher-notindex':set(),"FP-level-index":set(), 'FP-higher-index':set(),
               'inconsistent-tool':set(), 'TN':set(),
               'FN':set(),'VP':set(), "truth-level":set(), "no-truth-level":set(),"total_unclassified":set(),'notruth':[]} #  at the level ,'taxNotfoundintool':[]
    

    for read_name, kraken_read_info in input_dic.items():
        if read_name not in dic_tax_truth:
            read_tpfp_dic['notruth'].append(read_name)
            continue
        tax_true = dic_tax_truth[read_name]
        if mode=="raw_kraken":
            tax_predicted = kraken_read_info[0]
        elif mode=="predicted_tax":
            tax_predicted = kraken_read_info
        else:
            raise ValueError("mode not supported")

        if tax_predicted==1:
            tax_predicted=2 # probably 1 has another meaning in find tax level, that's fine the level should be much deeper 
    
        tax_true_level = find_tax_level(info,tree_df,parents, tax_true, tax_level)
        tax_predicted_level = find_tax_level(info,tree_df,parents, tax_predicted, tax_level)
    
        if tax_predicted_level==-1 and tax_predicted!=0:
            read_tpfp_dic['inconsistent-tool'].add(read_name)
            # tid_tool_taxlevel==-1 we are ignoring these reads if the tax is not in the node.dump 
            # we think these cases are limited and probably happened due to different node.dump in build vs evaluation 
    
        if tax_predicted==0 :
            read_tpfp_dic["total_unclassified"].add(read_name)
        if tax_predicted==-1 or tax_true_level < 1:
            logging.debug("to check "+str(tax_predicted)+" "+str(tax_true)+" "+str(tax_true_level))

            #to check  9400 632 0 -1
    
        if tax_true_level ==1 or  tax_true_level==-1 or (tax_predicted_level==-1 and tax_predicted!=0): # true is not at the level, is higher than the level     or true doest not exist in info
            #including 'inconsistent-tool'
            read_tpfp_dic["no-truth-level"].add(read_name)
            
        else:
            read_tpfp_dic["truth-level"].add(read_name)
    
        if tax_true_level >1 and tax_predicted==0:
            if tax_true in tax_index:
                read_tpfp_dic['FN'].add(read_name) # unclassified, among reads with true label at the level 
            else:
                read_tpfp_dic['TN'].add(read_name) # unclassified, among reads with true label at the level 
            
        if tax_true_level>1 and tax_predicted_level==tax_true_level :
            read_tpfp_dic['TP'].add(read_name) # correct
    
        if tax_true_level>1 and tax_predicted>1 and tax_predicted_level==1 :  # tool labled at higher than taxlevel 
            tool_level= tree_df[4][info[tax_predicted]]
            tax_true_toollevel = find_tax_level(info,tree_df,parents, tax_true, tool_level)
            if tax_true_toollevel == tax_predicted: # at the tool level is correct 
                read_tpfp_dic['VP'].add(read_name) # vague positive, one deeper lvel (ancestror)  match    
            if tax_true_toollevel != tax_predicted:
                if tax_true in tax_index:
                    read_tpfp_dic['FP-higher-index'].add(read_name) # tool labeld at higher level and worng (not a VP) 
                else:
                    read_tpfp_dic['FP-higher-notindex'].add(read_name) # tool labeld at higher level and worng (not a VP) 
    
        if tax_true_level > 1 and tax_predicted_level>1 and  tax_predicted_level != tax_true_level :
            if tax_true in tax_index:
                read_tpfp_dic['FP-level-index'].add(read_name) # tool labeled at the tax level (or lower) and wrong 
            else:
                read_tpfp_dic['FP-level-notindex'].add(read_name) # tool labeled at the tax level (or lower) and wrong             
               
    return read_tpfp_dic

In [ ]:
input_dic= kraken_kmers_cases['k31']
tax_level='species'
read_tpfp_dic= calculate_tp_fp("raw_kraken",input_dic,dic_tax_truth,info,tree_df,parents,tax_level,tax_index)
read_tpfp_dic.keys()


2025-08-09 16:34:46,934 - DEBUG - to check 2 386490 -1
2025-08-09 16:34:46,935 - DEBUG - to check 2 386490 -1
2025-08-09 16:34:46,935 - DEBUG - to check 0 386490 -1
2025-08-09 16:34:47,845 - DEBUG - to check 562 2778655 -1
2025-08-09 16:34:47,846 - DEBUG - to check 1446707 2778655 -1
2025-08-09 16:34:47,846 - DEBUG - to check 1134687 2778655 -1
2025-08-09 16:34:48,715 - DEBUG - to check 379 3024248 -1
2025-08-09 16:34:48,716 - DEBUG - to check 379 3024248 -1


dict_keys(['TP', 'FP-level-notindex', 'FP-higher-notindex', 'FP-level-index', 'FP-higher-index', 'inconsistent-tool', 'TN', 'FN', 'VP', 'truth-level', 'no-truth-level', 'total_unclassified', 'notruth'])

In [64]:
'NZ_RREV01000078.1__1_1' in read_tpfp_dic['FN']

True

In [ ]:
def calculate_true_k(kraken_kmers_cases,dic_tax_truth,info,tree_df,parents,tax_level,tax_index):
    tp_cases_dic={}
    for case in cases[4:5]:
        logging.debug('finding true k for case '+case)
        #merged  = _utils_kraken.read_kraken_classification(cases, truth_file, classification_folder )    
        input_dic= kraken_kmers_cases[case]
        tax_level='species'
        read_tpfp_dic_case= calculate_tp_fp("raw_kraken",input_dic,dic_tax_truth,info,tree_df,parents,tax_level,tax_index)
        logging.debug("Number of reads in the case TP : "+str(len(read_tpfp_dic_case['TP'])))
        logging.debug("Number of reads in the case not truth level : "+str(len(read_tpfp_dic_case['notruth'])))
        tp_cases_dic[case]=read_tpfp_dic_case['TP']

    true_k={}
    for read_name in read_names_list:
        true_k[read_name]=set()
        for case_k, case in  enumerate(cases[4:5]): # reversed(
            k = int(case[1:3])
            if read_name in tp_cases_dic[case]:
                true_k[read_name].add(k)
    logging.debug("Number of reads in the case TP : "+str(len(true_k)))
    logging.debug("Number of kmer sizes in the TP for read"+read_name+" : "+str(len(true_k[read_name])))
    return true_k


true_k = calculate_true_k(kraken_kmers_cases,dic_tax_truth,info,tree_df,parents,tax_level,tax_index)

k25


2025-08-09 16:45:31,039 - DEBUG - to check 2921447 386490 -1
2025-08-09 16:45:31,040 - DEBUG - to check 186817 386490 -1
2025-08-09 16:45:31,041 - DEBUG - to check 0 386490 -1
2025-08-09 16:45:31,939 - DEBUG - to check 562 2778655 -1
2025-08-09 16:45:31,940 - DEBUG - to check 1446707 2778655 -1
2025-08-09 16:45:31,941 - DEBUG - to check 573 2778655 -1
2025-08-09 16:45:32,807 - DEBUG - to check 2613769 3024248 -1
2025-08-09 16:45:32,808 - DEBUG - to check 1981173 3024248 -1
2025-08-09 16:45:34,501 - DEBUG - Number of reads in the case TP : 152138
2025-08-09 16:45:34,502 - DEBUG - Number of reads in the case not truth level : 194
2025-08-09 16:45:34,868 - DEBUG - Number of reads in the case TP : 249069
2025-08-09 16:45:34,869 - DEBUG - Number of kmer sizes in the TP for readNZ_CP028392.1__1_2 : 1


In [ ]:
len(true_k)

249069

In [ ]:

def find_true_ksize(cases,merged,info,tree_df,parents,tax_index, readids_max, level='species'):
    tp_cases_dic={}
    for case in cases:
        case1= 'taxon_tool_'+case
        tp_cases_dic[case]= calculate_tp_fp(merged,info,tree_df,parents,level,case1,tax_index) #+case
    print(len(tp_cases_dic),len(tp_cases_dic[case]))

    return tp_cases_dic, true_k

194

In [ ]:
# a= "C	NZ_OZ243912.1__1_1	1885344	1253	0:557 1885344:2 0:81 1885344:7 0:27 1885344:2 0:28 1885344:3 0:121 1885344:13 0:49 1885344:8 0:35 1885344:6 0:68 1885344:27 0:133 1885344:7 0:49"

# kraken_file="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/k31_out"
# kraken_file_out="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/k31_out_test"
# file_out=open(kraken_file_out,'w')
# file_in=open(kraken_file,'r')
# dic_kraken={}
# for line in file_in: 
#     line_split=line.strip().split("\t")
#     classified ,read_id, tax_krak, read_len, tax_kmers = line_split
#     if read_id in readids_max:
#         file_out.write(line+"\n")
# file_in.close()
# file_out.close()








In [95]:
kraken_file="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/max15/k31_out"
file_o=open(kraken_file,'r')
dic_kraken={}
cnt=0
for line in file_o:
    cnt+=1
    line_split=line.strip().split("\t")
    classified ,read_id, tax_krak, read_len, tax_kmers = line_split

In [96]:
cnt

86202

In [98]:
#workingdir
import pickle
model_folder="/vast/blangme2/smajidi5/metagenomics/metakpick_project/results/"# workingdir+"results/"
model_files =[ model_file for  model_file in os.listdir(model_folder) if model_file.endswith(".pkl") and model_file.startswith("Random_Forest_regression_models")]
model_files.sort()
model_file=model_files[-1]
#model_time_stamp=model_file.split("_")[1] # model_file=workingdir+"Random_Forest_regression_models"+model_time_stamp+".pkl" 
logging.info("Loading the model: "+model_file+" in the folder: "+model_folder)
loaded_regression_dic= pickle.load(open(model_folder+model_file, "rb"))
logging.info("Model loaded"+str(loaded_regression_dic))


2025-08-09 19:41:24,555 - INFO - Loading the model: Random_Forest_regression_models20250809_192205.pkl in the folder: /vast/blangme2/smajidi5/metagenomics/metakpick_project/results/
2025-08-09 19:41:25,188 - INFO - Model loaded({'k21': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k23': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k25': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k27': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k29': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k31': RandomForestRe

In [100]:
loaded_regression_dic[0]

{'k21': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                       n_jobs=20, random_state=14, verbose=True),
 'k23': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                       n_jobs=20, random_state=14, verbose=True),
 'k25': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                       n_jobs=20, random_state=14, verbose=True),
 'k27': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                       n_jobs=20, random_state=14, verbose=True),
 'k29': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                       n_jobs=20, random_state=14, verbose=True),
 'k31': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                       n_jobs=20, random_state=14, verbose=True)}

In [101]:
loaded_regression_dic[1]

IndexError: tuple index out of range